In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import tensorflow as tf
from functools import partial

from watch_recognition.augmentations import view_image
from watch_recognition.data_preprocessing import load_keypoints_data_as_kp

In [ ]:
%matplotlib inline

In [ ]:
X, y,_ = load_keypoints_data_as_kp(
    Path("../download_data/keypoints/train"),
)
X.shape, y.shape

In [ ]:
from watch_recognition.augmentations import set_shapes, encode_keypoints_to_mask, \
    add_sample_weights

dataset = tf.data.Dataset.from_tensor_slices((X, y))

AUTOTUNE = tf.data.experimental.AUTOTUNE


# dataset = dataset.map(
#         process_kp_data,
#         num_parallel_calls=AUTOTUNE,
# )

dataset = dataset.map(
        partial(encode_keypoints_to_mask, image_size=(224, 224, 3), mask_size=(28,28), extent=(1,1)),
        num_parallel_calls=AUTOTUNE
)
dataset = dataset.map(
    partial(set_shapes, img_shape=(224, 224, 3), target_shape=(28,28, 1)),
    num_parallel_calls=AUTOTUNE).batch(32).prefetch(AUTOTUNE)
dataset = dataset.map(add_sample_weights)

In [ ]:
view_image(dataset)